In [ ]:
import pyodbc
import paramiko
import pandas as pd
import datetime
import getpass
import xlsxwriter
from tqdm.notebook import tqdm
import win32com.client
import time
UID=input("Enter userid ")
PWD=getpass.getpass("Enter password ")
# Connecting to Netezza
conn1 = pyodbc.connect("DRIVER={NetezzaSQL};SERVER=10.11.45.135; PORT=5480;DATABASE=NHIPDHETLSANDBOX; UID="+UID+";PWD="+PWD+";",autocommit=True)
cur1=conn1.cursor()

today = datetime.date.today()
first = today.replace(day=1)
cur_mmyy=first.strftime("%b_%Y")
src_path = r"C:\Users\\"+UID+"\Downloads\\"
oldw_path="/OLABS_QA/prod/NHI/mnthly/Data_Profile/"

# To connect to putty
ssh_client=paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh_client.connect(hostname='10.11.45.9',username='{}'.format(UID),password='{}'.format(PWD))
# Executing command in OLDW server to get live and non_live db names
stdin,stdout,stderr=ssh_client.exec_command("grep -im 1 nhibdbname /etl_olab4/prd/olab/NHI/scripts/OLAB.Master.Environment.Setup.sh")
out=stdout.readlines()
nhi_non_livedb=out[0].split('"')[1]
stdin,stdout,stderr=ssh_client.exec_command("grep -im 1 nhiSourcebdbName /etl_olab4/prd/olab/NHI/scripts/OLAB.Master.Environment.Setup.sh")
out=stdout.readlines()
nhi_livedb=out[0].split('"')[1]
databases=[nhi_non_livedb,nhi_livedb]
tables=[]
# Reading the list of tables from the file in OLDW server
stdin,stdout,stderr=ssh_client.exec_command("cat /OLABS_QA/prod/NHI/mnthly/Data_Profile/data_profile_tables.txt")
output=stdout.readlines()
for val in output:
    tables.append(val.strip())

#Getting VBA file from server to local

def getvba():
    remotefile=oldw_path+"data_profile.bin"
    local_vba=  r"C:\Users\\"+UID+"\Downloads\data_profile.bin"
    sftp = ssh_client.open_sftp()
    try:
        sftp.get(remotefile,local_vba)
        sftp.close()
        print("VBA file saved in local")
    except Exception as e:
        print("Getting VBA file to Local failed")
        print("Error :")
        print(e)
getvba()
    
def file_processing(final_file):
    xl=win32com.client.Dispatch("Excel.Application")
    print("Running macros")
    file='Data_Profile_'+cur_mmyy+'.xlsm'
    oldw_final_file=oldw_path+file
    wb=xl.Workbooks.Open(Filename=final_file)
    try:
        xl.Visible=True
        xl.Application.Run(file+"!Module1.data_profile")

        wb.Close(SaveChanges=1)
        print("Success! "+file+" saved in Downloads folder")
        transfer(oldw_final_file,final_file)
    except Exception as err:
        print("failed")
        print(err)
        xl.Workbooks(1).Close(SaveChanges=0)
        xl.Application.Quit()
    xl.Quit()

#Transferring excel file to OLDW

def transfer(oldw_final_file,final_file):
    sftp = ssh_client.open_sftp()
    sftp.put(final_file,oldw_final_file)
    sftp.close()
    print("File transferred successfully to OLABS server")
    send_mail(oldw_final_file)

#Sending file over mail
def send_mail(file):
    sub="Data_Profile"
    stdin,stdout,stderr=ssh_client.exec_command("sh /OLABS_QA/scripts/SOPmetricsmail.sh {} {}".format(sub,file))
    out=stderr.readlines()
    if(len(out)==0):
        print("Report sent over mail successfully")
    else:
        print("Report mail failed")
        print(out)

def main_code():
    filename=r'C:\\Users\\'+UID+'\\Downloads\\Data_Profile_'+cur_mmyy+'.xlsm'
    with xlsxwriter.Workbook(filename) as workbook:
        workbook.add_vba_project(r'C:\Users\\'+UID+'\Downloads\data_profile.bin')
        
        for t in tqdm(tables):
            start_time = time.time()
            worksheet = workbook.add_worksheet(name=t)
            list1=[]
            list2=[]
            
            # Getting column names and other details from QA_COLUMNS tables
            for database,lst in zip([nhi_non_livedb,nhi_livedb],[list1,list2]):                
                line=0
                query1="select COLUMN_NUMBER,DATABASE_NAME,OBJECT_NAME,COLUMN_NAME,COLUMN_TYPE from system..qa_columns A where database_name='{}' AND OBJECT_NAME='{}' ORDER BY COLUMN_NUMBER".format(database,t)
                cur1.execute(query1)
                
                for i in cur1:
                    lst.append(list(i))
                try:
                    count_query="SELECT COUNT(*) FROM {}..{}".format(database,t)
                    cur1.execute(count_query)
                    for i in cur1:
                        tab_count=i[0]
                except pyodbc.Error as err:
                    print("Error occurred while getting table count")
                    print(err)
                    tab_count=0
                for i in lst:
                    column=i[3]
                    # Printing the process
                    print("Processing "+"\033[1m"+database+"\033[0m"+" table "+"\033[1m"+t+"\033[0m"+" column "+"\033[1m"+column+"\033[0m",end="\r")
                    # Running main query to get details of each column
                    query2='''SELECT {table_count} as TotalCnt,
                    SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) AS NullCnt,
                    SUM(CASE WHEN TRIM({col}) =' ' THEN 1 ELSE 0 END) AS BlankCnt,
                    TotalCnt-NullCnt AS NotNullCnt,
                    CAST(MIN({col}) AS VARCHAR(100)) as MinVal,
                    CAST(MAX({col}) AS VARCHAR(100)) AS MaxVal,
                    COUNT(DISTINCT ({col})) AS DistinctValCnt,
                    CAST((NotNullCnt-BlankCnt)*100/TotalCnt AS NUMERIC(5,2)) AS DATA_AVAILABILITY
                    FROM {dbase}..{tab}'''.format(dbase=database,tab=t,col=column,table_count=tab_count)
                    try:
                        cur1.execute(query2)
                        for i in cur1:
                            final=list(i)
                    except pyodbc.Error as ex:
                        print("Error occured while getting details of {} column".format(column))
                        print(ex)
                        final=[""]*11
                    lst[line]=lst[line]+final
                    line+=1
            columns1=['COLUMN_NUMBER','NONLIVE DATABASE_NAME','TABLE_NAME','COLUMN_NAME','COLUMN_TYPE']
            columns2=['NONLIVE TOTALCNT','LIVE TOTALCNT','NONLIVE NULLCNT','LIVE NULLCNT','NONLIVE BLANKCNT','LIVE BLANKCNT','NONLIVE NOTNULLCNT','LIVE NOTNULLCNT','NONLIVE MINVAL','LIVE MINVAL','NONLIVE MAXVAL','LIVE MAXVAL','NONLIVE DISTINCTVALCNT','LIVE DISTINCTVALCNT','NONLIVE DATA_AVAILABILITY','LIVE DATA_AVAILABILITY']

            columns=columns1+columns2
            
            list3=[]
            cols_len=len(columns1)
            for i,j in zip(list1,list2):
                meta=i[:cols_len]
                list4=[]
                for i1,j1 in zip(i[cols_len:],j[cols_len:]):
                    list4.append(i1)
                    list4.append(j1) 
                list3.append(meta+list4)
            
            worksheet.write_row(0,0,columns)
            for i,j in enumerate(list3,1):
                worksheet.write_row(i,0,j)
            end_time = time.time()
            print("\nTime took for table {}".format(t),end_time-start_time)
    file_processing(filename)
main_code()